<a href="https://colab.research.google.com/github/BoraShruti/VisualTransformersMNIST/blob/main/visual_transformers_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Transformer Experiments with WandB Integration

This notebook demonstrates how to integrate [Weights & Biases (WandB)](https://wandb.ai/) with a Vision Transformer (ViT) training pipeline. We will experiment with the following hyperparameters:

- **Patch Size:** Fixed at 4×4
- **Number of ViT Layers:** 4 to 8
- **Number of Attention Heads:** 2 to 4

For each configuration, the model will be trained and the test accuracy will be recorded and logged to WandB. Detailed explanations are provided in each section so that you can understand the purpose of each block of code.

In [ ]:
# Install wandb if it is not already installed
!pip install wandb
!pip install --upgrade wandb


## Importing Libraries

In this cell we import all the necessary libraries including TensorFlow for building our model, WandB for experiment tracking, and other useful libraries for data processing and result analysis.

In [ ]:
import wandb
from wandb.integration.keras import WandbCallback
import itertools
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models

# For reproducibility
import numpy as np
np.random.seed(42)
tf.random.set_seed(42)

## Dataset Preparation

In this section we load and preprocess the dataset. For demonstration purposes, we are using the MNIST dataset. In your project, replace this section with your actual data loading and preprocessing steps. The dataset is split into training, validation, and test sets.

In [ ]:
# Example: Loading the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Expand dims to add a channel dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Create a validation set from the training data
val_split = 0.1
val_size = int(len(x_train) * val_split)
x_val = x_train[:val_size]
y_val = y_train[:val_size]
x_train = x_train[val_size:]
y_train = y_train[val_size:]

# Create TensorFlow datasets
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Model Definition

Below is the definition of a helper function `create_vit_model` that constructs a simple Vision Transformer (ViT) model. The function takes the patch size, number of transformer layers, and number of attention heads as parameters. You can customize the model architecture as needed.

In [ ]:
def create_vit_model(patch_size, num_layers, num_heads, image_size=28, num_classes=10):
    """
    Creates a simple Vision Transformer (ViT) model.

    Parameters:
        patch_size (int): Size of each patch (patch_size x patch_size).
        num_layers (int): Number of transformer layers.
        num_heads (int): Number of attention heads in each transformer layer.
        image_size (int): Size of the input image (assumed to be square).
        num_classes (int): Number of classes for classification.

    Returns:
        model (tf.keras.Model): A compiled Vision Transformer model.
    """
    # Calculate the number of patches
    num_patches = (image_size // patch_size) ** 2

    # Input layer
    inputs = layers.Input(shape=(image_size, image_size, 1))

    # Create patches using a convolution layer
    patches = layers.Conv2D(filters=64, kernel_size=patch_size, strides=patch_size, padding='valid')(inputs)
    x = layers.Reshape((num_patches, 64))(patches)

    # Add positional embeddings
    positions = tf.range(start=0, limit=num_patches, delta=1)
    pos_embedding = layers.Embedding(input_dim=num_patches, output_dim=64)(positions)
    x = x + pos_embedding

    # Transformer encoder blocks
    for _ in range(num_layers):
        # Layer normalization
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Multi-head attention
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=64)(x1, x1)
        x2 = layers.Add()([x, attention_output])

        # MLP block with residual connection
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        mlp_output = layers.Dense(128, activation='relu')(x3)
        mlp_output = layers.Dense(64)(mlp_output)
        x = layers.Add()([x2, mlp_output])

    # Classification head
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

## Experiment Setup with WandB

In this section we configure our hyperparameter experiments using WandB. The experiment will use the following settings:

- **Patch Size:** Fixed at 4×4
- **Number of ViT Layers:** 4 to 8
- **Number of Attention Heads:** 2 to 4

For each combination, a new WandB run is initiated, the model is created, compiled, and trained. After training, the model is evaluated on the test set, and the test accuracy is logged to WandB and stored locally for summary.

In [ ]:
# Set the fixed patch size and define the range for the number of layers and heads
patch_size = 4  # 4x4 patches
layer_options = range(4, 9)  # Experiment with 4 to 8 layers
head_options = range(2, 5)   # Experiment with 2 to 4 heads

# Number of epochs for training
EPOCHS = 5  # Adjust the number of epochs as needed

# List to store experiment results
experiment_results = []

import time

for num_layers, num_heads in itertools.product(layer_options, head_options):
    # Initialize a new WandB run for the current experiment
    wandb.init(project="ViT_experiment",
               config={
                   "patch_size": f"{patch_size}x{patch_size}",
                   "num_layers": num_layers,
                   "num_heads": num_heads
               }, reinit=True)
    config = wandb.config

    print(f"Running experiment with patch_size: {config.patch_size}, num_layers: {config.num_layers}, num_heads: {config.num_heads}")

    # Create the Vision Transformer model with the current configuration
    model = create_vit_model(patch_size=patch_size,
                             num_layers=num_layers,
                             num_heads=num_heads,
                             image_size=28,
                             num_classes=10)

    # Compile the model with an appropriate optimizer and loss function
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model. The WandbCallback will automatically log training metrics to WandB
    history = model.fit(
      train_dataset,
      validation_data=val_dataset,
      epochs=EPOCHS,
      callbacks=[WandbCallback(save_graph=False,save_model=False,data_type="image")],  # Disable automatic model saving
      verbose=1
    )


    # Evaluate the trained model on the test set
    test_loss, test_acc = model.evaluate(test_dataset)
    print(f"Test Accuracy: {test_acc}")

    # Log the test accuracy to WandB
    wandb.log({"test_accuracy": test_acc})

    # Store the experiment results
    experiment_results.append({
        "patch_size": config.patch_size,
        "num_layers": config.num_layers,
        "num_heads": config.num_heads,
        "test_accuracy": test_acc
    })

    # Finish the current WandB run
    wandb.finish()

    # Optional: Pause briefly between experiments
    time.sleep(1)

# Convert the results into a DataFrame and display the summary
df_results = pd.DataFrame(experiment_results)
print("Summary of experiments:")
print(df_results)

Running experiment with patch_size: 4x4, num_layers: 4, num_heads: 2
Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - accuracy: 0.8119 - loss: 0.6058

wandb: WARNING No validation_data set, pass a generator to the callback.


844/844 ━━━━━━━━━━━━━━━━━━━━ 270s 283ms/step - accuracy: 0.8120 - loss: 0.6055 - val_accuracy: 0.9295 - val_loss: 0.2304
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - accuracy: 0.9617 - loss: 0.1252

wandb: WARNING No validation_data set, pass a generator to the callback.


844/844 ━━━━━━━━━━━━━━━━━━━━ 260s 281ms/step - accuracy: 0.9618 - loss: 0.1252 - val_accuracy: 0.9418 - val_loss: 0.1932
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - accuracy: 0.9711 - loss: 0.0940

...

Running experiment with patch_size: 4x4, num_layers: 7, num_heads: 2
Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.7938 - loss: 0.6689

wandb: WARNING No validation_data set, pass a generator to the callback.


844/844 ━━━━━━━━━━━━━━━━━━━━ 459s 513ms/step - accuracy: 0.7939 - loss: 0.6685 - val_accuracy: 0.9425 - val_loss: 0.1846
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step - accuracy: 0.9614 - loss: 0.1253

wandb: WARNING No validation_data set, pass a generator to the callback.


844/844 ━━━━━━━━━━━━━━━━━━━━ 428s 507ms/step - accuracy: 0.9614 - loss: 0.1253 - val_accuracy: 0.9552 - val_loss: 0.1397
Epoch 3/3
204/844 ━━━━━━━━━━━━━━━━━━━━ 5:08 482ms/step - accuracy: 0.9641 - loss: 0.1081

In [ ]:
!pip install wandb --upgrade


## Summary

Each experiment has been logged to WandB, and the test accuracies for the various configurations are printed at the end. You can inspect the detailed logs and visualizations on your WandB project page. This notebook provides a complete pipeline from data loading to model evaluation with extensive documentation to help you understand each step.